In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
pip install DeepMIMOv3 umap-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
#LWM을 하기위한 라이브러리 가져오기
import DeepMIMOv3
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt

plt . rcParams [ 'figure.figsize' ]  =  [ 12 ,  8 ]  # 기본 플롯 크기 설정

In [5]:
## Load and print the default parameters
# bandwith: 0.05GHz(50MHz 대역폭 사용)

parameters = DeepMIMOv3.default_params()
pprint(parameters)

{'OFDM': {'RX_filter': 0,
          'bandwidth': 0.05,
          'selected_subcarriers': array([0]),
          'subcarriers': 512},
 'OFDM_channels': 1,
 'active_BS': array([1]),
 'bs_antenna': {'FoV': array([360, 180]),
                'radiation_pattern': 'isotropic',
                'rotation': array([0, 0, 0]),
                'shape': array([8, 4]),
                'spacing': 0.5},
 'dataset_folder': './Raytracing_scenarios',
 'dynamic_scenario_scenes': array([1]),
 'enable_BS2BS': 1,
 'enable_doppler': 0,
 'enable_dual_polar': 0,
 'num_paths': 5,
 'scenario': 'O1_60',
 'ue_antenna': {'FoV': array([360, 180]),
                'radiation_pattern': 'isotropic',
                'rotation': array([0, 0, 0]),
                'shape': array([4, 2]),
                'spacing': 0.5},
 'user_rows': array([1]),
 'user_subsampling': 1}


In [6]:
## Change parameters for the setup
# Scenario O1_60 extracted at the dataset_folder
#LWM 동적 시나리오 불러오기
#자신의 LWM 파일 위치 경로 작성
# parameters['dataset_folder'] = r'/content/drive/MyDrive/Colab Notebooks/LWM'
parameters['dataset_folder'] = r'C:\Users\dlghd\졸업프로젝트\LWM'

# scnario = 02_dyn_3p5 <- 다운받은 파일(동적시나리오)
parameters['scenario'] = 'O2_dyn_3p5'
parameters['dynamic_scenario_scenes'] = np.arange(1000) #scene 0~9

# 각 사용자-기지국 채널에 대해 최대 10개 멀티패스 경로 사용
parameters['num_paths'] = 10

# User rows 1-100
parameters['user_rows'] = np.arange(100)

# Activate only the first basestation
parameters['active_BS'] = np.array([1])

parameters['activate_OFDM'] = 1

parameters['OFDM']['bandwidth'] = 0.05 # 50 MHz
parameters['OFDM']['subcarriers'] = 512 # OFDM with 512 subcarriers
parameters['OFDM']['selected_subcarriers'] = np.arange(0, 64, 1)
#parameters['OFDM']['subcarriers_limit'] = 64 # Keep only first 64 subcarriers

parameters['ue_antenna']['shape'] = np.array([1, 1]) # Single antenna
parameters['bs_antenna']['shape'] = np.array([1, 32]) # ULA of 32 elements
#parameters['bs_antenna']['rotation'] = np.array([0, 30, 90]) # ULA of 32 elements
#parameters['ue_antenna']['rotation'] = np.array([[0, 30], [30, 60], [60, 90]]) # ULA of 32 elements
#parameters['ue_antenna']['radiation_pattern'] = 'isotropic'
#parameters['bs_antenna']['radiation_pattern'] = 'halfwave-dipole'

In [7]:
print(parameters)

{'dataset_folder': 'C:\\Users\\dlghd\\졸업프로젝트\\LWM', 'scenario': 'O2_dyn_3p5', 'dynamic_scenario_scenes': array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160,

In [8]:
## Generate and inspect the dataset
import time
start = time.time()
dataset = DeepMIMOv3.generate_data(parameters)
end = time.time()
print(f"걸리는 시간: {round(end-start,2)}")

The following parameters seem unnecessary:
{'activate_OFDM'}

Scene 1/1000

Basestation 1

UE-BS Channels


Generating channels: 100%|█████████████████████████████████████████████████████| 69040/69040 [00:12<00:00, 5479.57it/s]



BS-BS Channels


Generating channels: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 332.99it/s]



Scene 2/1000

Basestation 1

UE-BS Channels


...


BS-BS Channels


Generating channels: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 500.33it/s]



Scene 39/1000

Basestation 1

UE-BS Channels


Reading ray-tracing: 100%|████████████████████████████████████████████████████| 69040/69040 [00:02<00:00, 31310.12it/s]


MemoryError: Unable to allocate 1.05 GiB for an array with shape (69040, 1, 32, 64) and data type complex64

In [ ]:
len(dataset)

In [ ]:
dataset[0]

# 사용자 접근 데이터

In [ ]:
user_data = dataset[0][0]['user']
print(user_data.keys())

# 사용자 채널 정보 확인

In [ ]:
# subcarries = 나눈 각각의 주파수 채널
# Channel = H <- 채널 벡터
# 채널 형태
# (user, UE antenna, Bs antenna, subcarrier)
channel = dataset[0][0]['user']['channel']
print(channel.shape)  

In [ ]:
print(dataset[0][0]['user']['channel'][100])

In [ ]:
print(len(dataset[0][0]['user']['channel'][100]))

In [ ]:
print(channel[10000][0][0])

# 사용자 위치 정보

In [ ]:
location = dataset[0][0]['user']['location']
print(location.shape)      # (사용자 수, 3)
print(location[0:4])         # 첫 번째 사용자의 (x, y, z)

# 경로정보

In [ ]:
paths = dataset[0][0]['user']['paths']
#사용자 수
print(len(paths))
# 첫 번째 사용자 경로 정보
print(paths[0])

# 기지국 정보

In [ ]:
bs_data = dataset[0][0]['basestation']
print(bs_data.keys())


# Scene 및 사용자 수

In [ ]:
for i, scene in enumerate(dataset[0]):
    user_locs = scene['user']['location']
    print(f"Scene {i}: {len(user_locs)} users")

# 채널 수

In [ ]:
len(dataset[0][0]['user']['channel'])

In [ ]:
print(dataset[0][0]['user']['paths'][0])

In [ ]:
scene = dataset[0][0] # scene 0
ue_idx = 0 # 첫 번째 사용자
channel = scene['user']['channel'][ue_idx]
print(channel.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

# channel CIR mat 정보 가져오기

In [ ]:
import scipy.io as sio

file_path = r'C:\Users\dlghd\졸업프로젝트\LWM\O2_dyn_3p5\scene_0\O2_dyn_3p5.1.CIR.mat'
mat_data = sio.loadmat(file_path)

# 파일 안의 key 확인
print(mat_data.keys())




In [ ]:
# 일반적으로 CIR key는 'CIR' 또는 'cir' 같은 이름일 가능성 높음
H_cir = mat_data['__header__']  
print(H_cir)

# Time-Prediction 시작
## Time Series 형태로 변환
### 단일사용자 채널 예측

In [ ]:
# H = dataset[0][0]['user']['channel'][100]  # (1, 32, 64), complex
# H = H.squeeze(0)  # → (32, 64)

# # 채널은 복소수 형태
# H_real = H.real   # shape: (32, 64)
# H_imag = H.imag   # shape: (32, 64)

# H_combined = np.concatenate([H_real, H_imag], axis=0)  # shape: (64, 64)


In [ ]:
channel_sequence = []

for scene in dataset:
    H = scene[0]['user']['channel'][100]
    H = H.squeeze(0)
    # 채널은 복소수 형태 
    H_real = H.real # (32, 64) = (BS antenna, subcarrier)
    H_imag = H.imag
    H_concat = np.concatenate([H_real, H_imag], axis=0) 
    channel_sequence.append(H_concat)

# T는 sence
channel_sequence = np.stack(channel_sequence)  # shape: (T, 64, 64) = (real/imag쌍, subcarrier)


In [ ]:
# 데이터 만들기
# seq_len = 시퀀스를 잘라서 사용
# [0,1,2,3,4], [1,2,3,4,5], [2,3,4,5]
seq_len = 5
X = []
y = []

for i in range(len(channel_sequence) - seq_len):
    input_seq = channel_sequence[i:i+seq_len-1] # t=0~3
    target = channel_sequence[i+seq_len-1] # t = 4
    
    X.append(input_seq)
    y.append(target)

# T = scene case
# samples = T - seq_len + 1
X = np.array(X) # shape: (samples, seq_len-1, 64, 64)
y = np.array(y) # shape: (samples, 64, 64)

In [ ]:
# Flatten
# 4096 = 64 * 64
X_flat = X.reshape(X.shape[0], X.shape[1], -1) # (5,4,4096) = (샘플, 시퀀스 길이)
y_flat = y.reshape(y.shape[0],-1) #(5,4096) # 샘플 대상 예측

In [ ]:
print(X_flat.shape)
print(y_flat.shape)

In [ ]:
# 데이터 생성
# 데이터 GPU로 계산하게 바꾸기
X_t = torch.tensor(X_flat, dtype = torch.float32)
y_t = torch.tensor(y_flat, dtype = torch.float32)
masked_pos = torch.full((X_t.shape[0], 1), seq_len - 2, dtype=torch.long, device='cuda')  # 마지막 위치 예측
X_t = X_t.to(device)
y_t = y_t.to(device)

In [ ]:
# 모델 초기화 및 gpu 이도 
# 손실 함수 및 옵티마이저 설정
from lwm_model import lwm

element_length = 64*64 # H_real * H_imag

model = lwm(element_length=element_length).to(device)
criterion =nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
# 모델 학습
import time
start = time.time()

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    optimizer.zero_grad()
    
    pred,_ = model(X_t, masked_pos)
    pred = pred.squeeze(1)
    
    loss = criterion(pred, y_t)
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")
end = time.time()

print(f"걸린시간: {round(end-start,2)}")

In [ ]:
# 모델 평가
model.eval()  

with torch.no_grad():  
    pred, _ = model(X_t, masked_pos)  # 새로운 데이터 넣기
    pred = pred.squeeze(1)

    val_loss = criterion(pred, y_t)
    print(f"Validation Loss: {val_loss.item():.6f}")


In [15]:
print(dataset[0][0]['user']['channel'])

[[[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   ...
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]]


 [[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   ...
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]]


 [[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   ...
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
   [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]]


 ...


 [[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j

In [53]:
# (예: 사용자 0번, 위치 index 100, scene 0~9까지)
# 같은 사용자의 같은 위치에서 시간에 따라 채널이 변하는지

ue_idx = 0
loc_idx = 100
channel_sequence = []

for scene in dataset:
    channel = scene[ue_idx]['user']['channel']
    h = channel[loc_idx, 0, :, 0]  # shape: (32,)
    channel_sequence.append(h)

channel_sequence = np.stack(channel_sequence)  # shape: (T, 32)


In [65]:
# 입출력 시퀀스
import numpy as np

#batch_size
seq_len = 5

X = []
y = []

for i in range(len(channel_sequence) - seq_len):
    # shape: (seq_len, features)
    # target: (features)
    input_seq = channel_sequence[i:i+seq_len]
    target = channel_sequence[i+seq_len]
    
    X.append(input_seq)
    y.append(target)



X = np.array(X)
y = np.array(y)
# 마스킹 위치 
masked_pos = torch.full((X.shape[0], 1), seq_len - 1, dtype=torch.long)

print("입력 시퀀스 X:", X.shape)
print("예측 대상 y:", y.shape)
print(masked_pos)

입력 시퀀스 X: (5, 5, 32)
예측 대상 y: (5, 32)
tensor([[4],
        [4],
        [4],
        [4],
        [4]])


In [55]:
# 데이터 텐서로 바꾸기
import torch

X_t = torch.tensor(X, dtype=torch.float32, device=device)  # shape: (samples, seq_len, element_length)
y_t = torch.tensor(y, dtype=torch.float32, device=device)  # shape: (samples, element_length)


In [56]:
# DataLoader로 묶기
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_t, y_t)
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True)

In [57]:
# LWM 모델 가져오기
from lwm_model import lwm
import torch.nn as nn
import torch.optim as optim

element_length = 64 # 32 imag채널 + 32 real채널

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = lwm(element_length=element_length).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

In [67]:
# 모델 학습
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        seq_len = X_batch.shape[1]
        masked_pos = masked_pos
        
        optimizer.zero_grad()
        pred, _ = model(X_batch, masked_pos)
        pred = pred.squeeze(1)
        
        loss = criterion(pred, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.6f}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (25x32 and 64x64)

In [22]:
print(channel_sequence)

[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
  0.+0.j 0.+0.j]]


In [23]:
print(dataset[0][0]['user']['paths'][0]['num_paths'])


0


In [24]:
print(dataset[0][0].keys())


dict_keys(['user', 'basestation', 'location'])


# LWM 동적시나리오 가져와서 분석해보기

In [62]:
channels = []

for scene_idx in range(10):  # scene 0~9
    user_channels = []

    for user in dataset[scene_idx]:  # 각 사용자
        channel = user['user']['channel']  # shape: (1891, 1, 32, 1)
        channel = channel.squeeze(1).squeeze(-1)  # shape: (1891, 32)
        user_channels.append(channel)

    user_channels = np.stack(user_channels)  # (num_users, 1891, 32)
    channels.append(user_channels)

# → channels: (10 scenes, num_users, 1891, 32)
channels = np.stack(channels)             # shape: (10, num_users, 1891, 32)
channels = channels.transpose(1, 0, 2, 3)  # → (num_users, 10, 1891, 32)

# 예: 특정 위치 index만 사용 (예: loc_idx = 100)
loc_idx = 100
channel_data = channels[:, :, loc_idx, :]  # shape: (num_users, 10, 32)

# 데이터 나누기
seq_len = 10
input_ids = torch.tensor(channel_data[:, :seq_len-1, :], dtype=torch.float32)
target_ids = torch.tensor(channel_data[:, seq_len-1, :], dtype=torch.float32)
masked_pos = torch.full((input_ids.shape[0], 1), seq_len-2, dtype=torch.long)


In [63]:
# 학습/테스트 분리
# train 80%, test 20%
split_idx = int(0.8*batch_size)

X_train  = input_ids[:split_idx]
y_train = target_ids[:split_idx]
pos_train = masked_pos[:split_idx]

X_test = input_ids[split_idx:]
y_test = target_ids[split_idx:]
pos_test = masked_pos[split_idx:]

NameError: name 'batch_size' is not defined

In [ ]:
#DataLoader 설정
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train, y_train, pos_train)
test_dataset = TensorDataset(X_test, y_test, pos_test)

train_loader = DataLoader(train_dataset, batch_size=32 , shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
# 학습 파이프라인
from lwm_model import lwm
import torch.nn as nn
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = lwm(element_length=element_length).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()


In [ ]:
# 학습 루프
n_epochs = 100


for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    count = 0

    for input_ids_batch, target_ids_batch, masked_pos_batch in train_loader:
        input_ids_batch = input_ids_batch.to(device)
        target_ids_batch = target_ids_batch.to(device)
        masked_pos_batch = masked_pos_batch.to(device)

        optimizer.zero_grad()

        logits, _ = model(input_ids_batch, masked_pos_batch)
        loss = criterion(logits.squeeze(1), target_ids_batch.squeeze(1))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        count += 1

    if epoch % 5 == 0:
      avg_loss = total_loss / count
      print(f"Epoch {epoch}: Avg Loss = {avg_loss:.6f}")


In [ ]:
# 예측 후 시각화
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad():
    pred, _ = model(input_ids.to(device), masked_pos.to(device))
    pred = pred.squeeze(1).cpu().numpy()
    target = target_ids.squeeze(1).numpy()

# 예: 사용자 0의 첫 번째 feature
plt.plot(pred[0], label="Predicted")
plt.plot(target[0], label="Actual", linestyle="--")
plt.title("Predicted vs Actual (User 0)")
plt.legend()
plt.grid(True)
plt.show()


# LWM을 가져와서 예측하기

In [60]:
## 데이터 준비 예시

import torch
import numpy as np

# 예시: shape (batch, seq_len, element_length)
# element_length = 채널 real + imag = 예: 64
# seq_len = 예: 10개 시점
# batch_size = 사용자 수

# batch_size가 커질수록 정확도 하락
seq_len = 10 # 총 10개 time step ex(scene 0 ~ 9)
element_length = 64 # 채널 real + imag

# dummy channel data (여기에 실제 채널 데이터 사용)
channels = []  # 전체 scene 채널을 모을 리스트

for scene_idx in range(10):  # scene 0 ~ 9
    user_channels = [] # 해당 scene의 모든 사용자 채널 저장

    # dataset[scene_idx]에서 한 사용자의 딕셔너리는
    # {'user': {...}, 'basestation': {...}} 구조
    for user_dict in dataset[scene_idx]:  # 각 scene의 사용자 순회
        # channel = user_dict['channel']  # ← 이것 때문에 KeyError 발생
        channel = user_dict['user']['channel']  # 실제 데이터 구조에 맞춰 수정

        # (1) shape 확인 후 squeeze
        # 보통 ( something, 1, something, 1 ) 형태라면 이렇게:
        channel = channel.squeeze(1).squeeze(-1)  # → (32,) 형태로 변형
        # (2) 사용자 채널 리스트에 누적
        user_channels.append(channel)

    # scene 하나(10개 중 1개)에 대한 "모든 사용자" 채널을 stack
    channels.append(np.stack(user_channels))  # (user, antenna) shape로 stack

# (scene, user, antenna) → (user, scene, antenna)
channel_data = np.stack(channels, axis=0).transpose(1, 0, 2)  # → (사용자 수, 10, 32)
batch_size = channel_data.shape[0]  # 사용자 수 기반으로 설정

# 아래코드는 dumy 채널 데이터였음
# channel_data = np.random.randn(batch_size, seq_len, element_length).astype(np.float32)

input_ids = torch.tensor(channel_data[:, :seq_len-1, :])       # t=0~(n-2)
target_ids = torch.tensor(channel_data[:, seq_len-1:seq_len, :])  # t=(n-1)
masked_pos = torch.full((batch_size, 1), seq_len-2)  # 마지막 step 위치


SyntaxError: expected ':' (3271643563.py, line 22)

In [29]:
print(type(user))

<class 'dict'>


In [30]:
print(user)

{'user': {'paths': array([{'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
       {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
       {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
       ...,
       {'num_paths': 1, 'DoD_phi': array([-4.67836], dtype=float32), 'DoD_theta': array([90.9207], dtype=float32), 'DoA_phi': array([175.322], dtype=float32), 'DoA_theta': array([89.0793], dtype=float32), 'phase': array([3.80264], dtype=float32), 'ToA': array([1.03828e-06], dtype=float32), 'LoS': array([0.], dtype=float32), 'power': array([4.792699e-10], dtype=float32)},
       {'num_paths': 1, 'DoD_phi': array([-4.67536], dtype=float32), 'DoD_theta': array([90.9201], dtype=float32), 'DoA_phi': array([175.325], dtype=float32), 'DoA_theta': array([89.07

In [32]:
print(dataset[0][0])

{'user': {'paths': array([{'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
       {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
       {'num_paths': 0, 'DoD_phi': [], 'DoD_theta': [], 'DoA_phi': [], 'DoA_theta': [], 'phase': [], 'ToA': [], 'power': [], 'LoS': []},
       ...,
       {'num_paths': 1, 'DoD_phi': array([-4.67836], dtype=float32), 'DoD_theta': array([90.9207], dtype=float32), 'DoA_phi': array([175.322], dtype=float32), 'DoA_theta': array([89.0793], dtype=float32), 'phase': array([3.80264], dtype=float32), 'ToA': array([1.03828e-06], dtype=float32), 'LoS': array([0.], dtype=float32), 'power': array([4.792699e-10], dtype=float32)},
       {'num_paths': 1, 'DoD_phi': array([-4.67536], dtype=float32), 'DoD_theta': array([90.9201], dtype=float32), 'DoA_phi': array([175.325], dtype=float32), 'DoA_theta': array([89.07